In [ ]:
import numpy as np
import copy
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, Sequential
from keras import regularizers
from keras import backend as K
import Data_prep as dp
from scipy.io import savemat, loadmat
from keras.losses import mse, binary_crossentropy
from keras.layers import Reshape, Lambda, Input, Dense, Flatten, Conv2D, Conv2DTranspose
from keras.layers import Activation, ZeroPadding2D, BatchNormalization, MaxPooling2D, AveragePooling2D, UpSampling2D
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
K.set_image_data_format('channels_last')
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import scipy.io
import numpy as np
from numpy import load
from numpy import zeros
from numpy import ones
from keras import backend
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from matplotlib import pyplot
from keras.constraints import Constraint
import tensorflow as tf

In [ ]:
def normalize(data):
    
    max_=np.max(data, axis=0)
    min_=np.min(data, axis=0)
    mean_=np.mean(data,axis=0)
    std_=np.std(data,axis=0)
    
    return np.divide((data-mean_),(std_))

In [ ]:
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(tf.math.multiply(y_true, y_pred))

class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}

def generator(latent_dim):

    #decoder network:
    l_decode=0.0
    #input/BC:
    latent_input = Input(shape=(latent_dim,), name='latent_decode') #input
    x = latent_input
    init = RandomNormal(stddev=0.02)
    #fully connected layer 1:
    x = Dense(1000, kernel_initializer=init, kernel_regularizer = regularizers.l1(l_decode))(x)
    x = BatchNormalization(axis = 1)(x)
    x = LeakyReLU(alpha=0.2)(x)

    #fully connected layer 2:
    x = Dense(5000, kernel_initializer=init, kernel_regularizer = regularizers.l1(l_decode))(x)
    x = BatchNormalization(axis = 1)(x)
    x = LeakyReLU(alpha=0.2)(x)

    #reshaping:
    x = Reshape((10, 125, 4))(x)

    #CNN/upsampling layer 1:
    x = Conv2DTranspose(4, (5, 5), strides = (1, 1), padding='same', kernel_initializer=init, kernel_regularizer = regularizers.l1(l_decode))(x)
    x = BatchNormalization(axis = 3)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D((2, 2), interpolation = 'bilinear')(x)
    #x = AveragePooling2D((2, 2), padding='same')(x)
    x = Conv2DTranspose(4, (5, 5), strides = (1, 1), padding='same', kernel_initializer=init, kernel_regularizer = regularizers.l1(l_decode))(x)
    x = BatchNormalization(axis = 3)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D((2, 2), interpolation = 'bilinear')(x)
    #x = AveragePooling2D((2, 2), padding='same')(x)
    #output:
#     x = Conv2D(1, (9, 9), padding='same', strides = (4, 4), kernel_initializer=init)(x)
#     x = BatchNormalization(axis = 3)(x)
#     x = LeakyReLU(alpha=0.2)(x)
    outputs = Conv2DTranspose(1, (9, 9), padding='same', strides = (1, 1), kernel_initializer=init, activation = 'tanh', name='bathymetry')(x)
    #outputs = UpSampling2D((2, 2))(x)
    #set decoder model:
    decoder = Model([latent_input], outputs, name='decoder')
    return decoder 


In [ ]:
def critic(in_shape=(40, 500, 1)):

    #decoder network:
    l_decode=0.0
    #input/BC:
    inputs = Input(shape=in_shape) #input
    const = ClipConstraint(100000)
    init = RandomNormal(stddev=0.1)
    y = inputs
    y = Conv2D(128, (5, 5), padding='same', strides=(2, 2))(y)
    y = BatchNormalization(axis = 3)(y)
    y = LeakyReLU(alpha=0.5)(y)
    y = MaxPooling2D((2, 2), padding='same')(y)
    y = Dropout(0.25)(y)
    y = Conv2D(128, (5, 5), padding='same', strides=(2, 2))(y)
    y = BatchNormalization(axis = 3)(y)
    y = LeakyReLU(alpha=0.5)(y)
    y = MaxPooling2D((2, 2), padding='same')(y)
    y = Dropout(0.25)(y)
    y = Conv2D(128, (5, 5), padding='same', strides=(2, 2))(y)
    y = BatchNormalization(axis = 3)(y)
    y = LeakyReLU(alpha=0.5)(y)
    y = MaxPooling2D((2, 2), padding='same')(y)
    y = Dropout(0.25)(y)
    y = Flatten()(y)
    y = Dense(1024)(y)
    y = BatchNormalization(axis = 1)(y)
    y = LeakyReLU(alpha=0.2)(y)
    y = Flatten()(y)
    y = Dense(512)(y)
    y = BatchNormalization(axis = 1)(y)
    y = LeakyReLU(alpha=0.2)(y)
    outputs = Dense(1, activation = 'sigmoid')(y)
    #y = BatchNormalization(axis = 1)(y)
    #outputs = LeakyReLU(alpha=0.2)(y)
    #set critic model:
    critic = Model([inputs], [outputs])
    opt = Adam(lr=0.0002, beta_1=0.5)
    critic.compile(loss='binary_crossentropy', optimizer=opt)
    return critic 


In [ ]:
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model(gen_output)
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise], [gan_output])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model


In [ ]:
# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X = images[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X], y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    #x_input = tf.random.normal(latent_dim * n_samples)
    z_input = tf.random.normal([n_samples, latent_dim]) 
    return [z_input]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images], y

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128, n_steps = 4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    d_loss1 = 0 
    d_loss2 = 0
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            real_iter = 1
            if (d_loss1 > 0):
                real_iter = 1
            fake_iter = 1
            if (d_loss2 > 0):
                real_iter = 1
            for k in range(real_iter):
                # get randomly selected 'real' samples
                X_real, y_real = generate_real_samples(dataset, n_batch)
                # update discriminator model weights
                d_loss1 = d_model.train_on_batch([X_real], y_real)
                # generate 'fake' examples
            for kk in range(real_iter):
                [X_fake], y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
                # update discriminator model weights
                d_loss2 = d_model.train_on_batch([X_fake], y_fake)
                # prepare points in latent space as input for the generator
                [z_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input], [y_gan])
            ####
            d_loss1 = d_model.test_on_batch([X_real], y_real)
            d_loss2 = d_model.test_on_batch([X_fake], y_fake)
            g_loss = gan_model.test_on_batch([z_input], [y_gan])
            ####
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))

In [ ]:
def train_c(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128, n_steps = 4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    d_loss1 = 0 
    d_loss2 = 0
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            print(i + j)
            real_iter = 1
            if (d_loss1 > 0):
                real_iter = 1
            fake_iter = 1
            if (d_loss2 > 0):
                real_iter = 1
            for k in range(real_iter):
                # get randomly selected 'real' samples
                X_real, y_real = generate_real_samples(dataset, n_batch)
                # update discriminator model weights
                d_loss1 = d_model.train_on_batch([X_real], y_real)
            # generate 'fake' examples
            for kk in range(real_iter):
                [X_fake], y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
                # update discriminator model weights
                d_loss2 = d_model.train_on_batch([X_fake], y_fake)
            # prepare points in latent space as input for the generator
            [z_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            ####
            d_loss1 = d_model.test_on_batch([X_real], y_real)
            d_loss2 = d_model.test_on_batch([X_fake], y_fake)
            ####
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2))

In [ ]:
def train_g(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128, n_steps = 4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # prepare points in latent space as input for the generator
            [z_input] = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            # create inverted labels for the fake samples
            g_loss = gan_model.train_on_batch([z_input], [y_gan])
            ####
            g_loss = gan_model.test_on_batch([z_input], [y_gan])
            ####
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f' %(i+1, j+1, bat_per_epo, g_loss))

In [ ]:
data = loadmat('Sav_BC_func_PCGA_1std_noise10_std4_zmax94_48_12_s0_5_6.mat')
vel = data['velocity_prof'] #velocities
z_f = data['z_f'] #free surface elevation
Q_b = data['Q_b'] #influx
Z = data['Z'] #bathymetry
vx, vy, _ = dp.xy_vel_sep(vel) #v_x/v_y separation
data_z = np.reshape(Z.T, (4500, 41, 501, 1))
data_z = data_z[:, :40, :500, :]
std_ = np.reshape(np.std(np.reshape(data_z, (4500, 20000)), axis = 0), (1,40,500,1))
mean_ = np.reshape(np.mean(np.reshape(data_z, (4500, 20000)), axis = 0), (1,40,500,1))
min_ = np.reshape(np.min(np.reshape(data_z, (4500, 20000)), axis = 0), (1,40,500,1))
max_ = np.reshape(np.max(np.reshape(data_z, (4500, 20000)), axis = 0), (1,40,500,1))
data_zn = ((data_z - min_) / (max_ - min_))
data_zn = (data_zn - 0.5) * 2

In [ ]:
g_model = generator(50)
c_model = critic()
gan_model = define_gan(g_model, c_model)
#train(g_model, c_model, gan_model, data_zn, 50, n_epochs=100, n_batch=128)

In [ ]:
epo = 100
for k in range(epo):
    #g_model = generator(50)
    num_sample = data_zn.shape[0]
    fake_samples = generate_fake_samples(g_model, 50, num_sample)[0][0]
    x_train = np.concatenate((fake_samples, data_zn))
    y_train = np.concatenate((np.zeros((num_sample, 1)), (np.ones((num_sample, 1)))))
    r_index = np.random.permutation(num_sample * 2) - 1
    x_train = x_train[r_index, :, :, :]
    y_train = y_train[r_index, :]
    n_epoch = 1
    for j in range(n_epoch):
        for i in range(282):
            c_model.train_on_batch(x_train[i * 16 : (i + 1) * 16], y_train[i * 16 : (i + 1) * 16])
            c_model.evaluate(x_train[i * 16 : (i + 1) * 16], y_train[i * 16 : (i + 1) * 16])
    train_g(g_model, c_model, gan_model, data_zn, 50, n_epochs=2, n_batch=128, n_steps = 10)

In [ ]:
c_model = critic()
opt = Adam(lr=0.0002, beta_1=0.5)
c_model.compile(loss='binary_crossentropy', optimizer=opt)
c_model.fit(x_train, y_train, validation_split=0.0)

In [ ]:
c_model.fit(x_train, y_train, validation_split=0.0)

In [ ]:
c_model.fit(data_zn, np.ones((4500, 1)), validation_split=0.0)

In [ ]:
c_model.fit(fake_samples, np.zeros((4500, 1)), validation_split=0.0)

In [ ]:
y_train.shape

In [ ]:
im = generate_real_samples(data_zn, 10)[0][0]
#plt.figure(figsize=(40,4))
plt.imshow(im[0,:,:,0], aspect='auto', vmin = -1, vmax = 1)
plt.colorbar()

In [ ]:
im = generate_fake_samples(g_model, 50, 10)[0][0]
im = ((im / 2) + 0.5) * (max_ - min_) + min_
plt.figure(figsize=(50,5))
plt.imshow(im[0,:,:,0], aspect='auto')
plt.colorbar()

In [ ]:
im = generate_fake_samples(g_model, 50, 10)[0][0]
im = ((im / 2) + 0.5) * (max_ - min_) + min_
fake_river = np.reshape(Z.T, (4500, 41, 501, 1))[0, :, :, 0]
fake_river[:40, :500] = im[0,:,:,0]
dp.plt_im_tri(depth=fake_river.ravel(), fig_title = 'fake_river', fig_name = 'fake_river_3.png')

In [ ]:
im = generate_real_samples(data_zn, 10)[0][0]
im = ((im / 2) + 0.5) * (max_ - min_) + min_
fake_river = np.reshape(Z.T, (4500, 41, 501, 1))[0, :, :, 0]
fake_river[:40, :500] = im[0,:,:,0]
dp.plt_im_tri(depth=fake_river.ravel(), fig_title = 'real_river', fig_name = 'real_river_3.png')

In [ ]:
g_model.save_weights('Bathy_Generator.h5')

In [ ]:
c_model.save_weights('Bathy_Critic.h5')

In [ ]:
c_model.evaluate(x_train, y_train)

In [ ]:
c_model(im)

In [ ]:
c_model(x_train[7000:7001,:,:,:])

In [ ]:
def sampling(input_param):
    """
    sampling the latent space from a Gaussian distribution:

    # Input
        input_param: mean and log of variance of q(z|x)

    # Output
        z: sampled latent space vector
    """
    
    #mean and log(var):
    z_mean, z_log_var = input_param 
    
    #dimensions:
    dim_1 = K.shape(z_mean)[0]
    dim_2 = K.int_shape(z_mean)[1]
    
    #sampling:
    norm_sample = K.random_normal(shape=(dim_1, dim_2))
    return z_mean + K.exp(0.5 * z_log_var) * norm_sample

In [ ]:
def sampling(input_param):
    """
    sampling the latent space from a Gaussian distribution:

    # Input
        input_param: mean and log of variance of q(z|x)

    # Output
        z: sampled latent space vector
    """
    
    #mean and log(var):
    z_mean, z_log_var = input_param 
    
    #dimensions:
    dim_1 = K.shape(z_mean)[0]
    dim_2 = K.int_shape(z_mean)[1]
    
    #sampling:
    norm_sample = K.random_normal(shape=(dim_1, dim_2))
    return z_mean + K.exp(0.5 * z_log_var) * norm_sample

In [ ]:
#encoder network:

#regularization coefficient:
l_encode=0.0

#input:
bathy_input = Input(shape=(41,501, 1), name = 'velocity')

#CNN/pooling layer 1:
x = Conv2D(2, (3, 3), strides=1, padding='same',kernel_regularizer = regularizers.l2(l_encode))(bathy_input)
x = BatchNormalization(axis = 3)(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

#CNN/pooling layer 2:
x = Conv2D(4, (3, 3), strides=1, padding='same',kernel_regularizer = regularizers.l2(l_encode))(x)
x = BatchNormalization(axis = 3)(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

#flattening:
shape = K.int_shape(x)
x = Flatten()(x)

#fully connected layer 1:
x = Dense(500,kernel_regularizer = regularizers.l2(l_encode))(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)

#output:
z_mean = Dense(50,activation='sigmoid',name='z_mean')(x)
z_log_var = Dense(50,activation='sigmoid',name='z_log_var')(x)
z = Lambda(sampling, output_shape=(50,), name='latent_encode')([z_mean, z_log_var])

#set encoder model:
encoder = Model(bathy_input, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

In [ ]:
#decoder network:

#regularization coefficient
l_decode=0.0

#input/BC:
BC_input = Input(shape=(2, ), name = 'BC') #BC
latent_input = Input(shape=(50,), name='latent_decode') #input
x = keras.layers.concatenate([latent_input, BC_input])

#fully connected layer 1:
x = Dense(500, kernel_regularizer = regularizers.l2(l_decode))(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)

#fully connected layer 2:
x = Dense((shape[1]-1)*(shape[2]-1)*shape[3], kernel_regularizer = regularizers.l2(l_decode))(x)
x = BatchNormalization(axis = 1)(x)
x = Activation('relu')(x)

#reshaping:
x = Reshape((shape[1]-1, shape[2]-1, shape[3]))(x)

#CNN/upsampling layer 1:
x = Conv2DTranspose(4, (3, 3), padding='same', kernel_regularizer = regularizers.l2(l_decode))(x)
x = BatchNormalization(axis = 3)(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)

#CNN/upsampling layer 2:
x = Conv2DTranspose(2, (3, 3), padding='same', kernel_regularizer = regularizers.l2(l_decode))(x)
x = BatchNormalization(axis = 3)(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)

#output:
outputs = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same', name='bathymetry')(x)

#set decoder model:
decoder = Model([latent_input,BC_input], outputs, name='decoder')
decoder.summary()

In [ ]:
vae_outputs = decoder([encoder(bathy_input)[2],BC_input])
vae = Model([bathy_input,BC_input], vae_outputs, name='vae')

In [ ]:

def vae_loss(y_true, y_pred):
    #KL coefficient:
    l_kl=1.0 
    
    #mse loss:
    mse_loss = mse(K.flatten(y_true), K.flatten(y_pred))
    
    #KL loss:
    kl_loss = - 0.5 * K.mean(1.0 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    
    #total loss
    vae_loss = mse_loss +l_kl*kl_loss
    return vae_loss

In [ ]:
vae.compile(optimizer='Adam', loss=vae_loss)
vae.summary()

In [ ]:
Z_dummy = Z.reshape(41,501,-1) 
Z_new = np.zeros((Z_dummy.shape[2],Z_dummy.shape[0],Z_dummy.shape[1])) 
for i in range(Z_dummy.shape[2]):
    Z_new[i,:,:] = Z_dummy[:,:,i]     
del Z_dummy

#check Z_new shape:
print(Z_new.shape)

In [ ]:
vx_dummy = vx.reshape(41,501,-1)
vx_new = np.zeros((vx_dummy.shape[2],vx_dummy.shape[0],vx_dummy.shape[1]))
for i in range(vx_dummy.shape[2]):
    vx_new[i,:,:] = vx_dummy[:,:,i]
del vx_dummy

#check vx_new shape:
print(vx_new.shape)

In [ ]:
vy_dummy = vy.reshape(41,501,-1)
vy_new = np.zeros((vy_dummy.shape[2],vy_dummy.shape[0],vy_dummy.shape[1]))
for i in range(vy_dummy.shape[2]):
    vy_new[i,:,:] = vy_dummy[:,:,i]
del vy_dummy

#check vy_new shape:
print(vy_new.shape)

In [ ]:
#add channel dimension:

y_train = Z_new.reshape(-1,41,501,1) #output (Z)
x_train = vx_new.reshape(-1,41,501,1) #input (vx; can be replaced with vy or both vx/vy)

#check shapes:
print(x_train.shape)
print(y_train.shape)

In [ ]:
#find normalizing parameters:

x_min = np.min(x_train, axis=0)
y_min = np.min(y_train, axis=0)
x_range = (np.max(x_train, axis=0)-np.min(x_train, axis=0))
y_range = (np.max(y_train, axis=0)-np.min(y_train, axis=0))

#check shapes:
print(x_min.shape)
print(y_min.shape)
print(x_range.shape)
print(y_range.shape)

In [ ]:
#normalize input/outputs:

x_train_norm = (x_train-x_min)/x_range 
y_train_norm = (y_train-y_min)/y_range 

#check shapes:
print(x_train_norm.shape)
print(y_train_norm.shape)

#check min/max:
print(np.min(x_train_norm))
print(np.min(y_train_norm))
print(np.max(x_train_norm))
print(np.max(y_train_norm))

In [ ]:
#normalize BC (ensure it's between 0 and 1):

BC = np.zeros((2, z_f.shape[1]))
BC[0,:] = z_f/120
BC[1,:] = Q_b/10000

#check shapes and maxes:
print(BC.shape)
print(max(BC[0,:]))
print(max(BC[1,:]))

In [ ]:
# data preparation:

#generate a seed number for reproducibility:
seed_num=np.random.randint(0,1e5)
#seed_num=0 #use for reproducing a previously known seed #
print(seed_num)

#generate seed:
np.random.seed(seed_num)

#shuffle based on the seed:
indx=np.arange(x_train_norm.shape[0]) #original indices (ascending) 
indx_new=copy.deepcopy(indx) #new, to be shuffled, indices
np.random.shuffle(indx_new) #shuffled idices

#create shuffled input/outputs:
x_train_norm_shfl=copy.deepcopy(x_train_norm)
y_train_norm_shfl=copy.deepcopy(y_train_norm)
BC_shfl=copy.deepcopy(BC)
for i in range(x_train_norm.shape[0]):
    x_train_norm_shfl[i,:,:,:]=x_train_norm[indx_new[i],:,:,:]
    y_train_norm_shfl[i,:,:,:]=y_train_norm[indx_new[i],:,:,:]
    BC_shfl[:,i]=BC[:,indx_new[i]]

In [ ]:
N=700 # number of data to be used
val_split=0.2 #validation split

#vae.load_weights('vae_i_m700_2_4_B_500_50_l0_1_0.h5') #to load saved weights, if necessary
vae.fit(x={'velocity':x_train_norm_shfl[:N,:,:,:], 'BC':BC_shfl[:,:N].T},y=y_train_norm_shfl[:N,:40,:500,:],
        epochs=100,batch_size=32,shuffle=False,validation_split=val_split)

In [ ]:
g_model.summary()